In [13]:
# Libraries
import pandas as pd
import numpy as np
from PIL import Image
import os

`Attribute exploration (ground truth labels for the images)`

Every image has annotations with ground truth values for the different attributes (312):

* A 312 vector with values ranging from 0-1 -> I have binarized them for clarity
*  Each of the values in the vector correspond to a defined attribute which I have also printed


In [50]:
# Load the attribute names from the attributes.txt file
with open('/Users/lucialarraona/Desktop/responsibleai/ConceptBottleneck/CUB/CUB_200_2011/attributes.txt', 'r') as f:
    attribute_names = [line.strip() for line in f.readlines()]

# Load the class attribute labels from the class_attribute_labels_continuous.txt file
class_attribute_labels = np.loadtxt('/Users/lucialarraona/Desktop/responsibleai/ConceptBottleneck/CUB/CUB_200_2011/CUB_200_2011/attributes/class_attribute_labels_continuous.txt')

# Load the class labels for the images
with open('/Users/lucialarraona/Desktop/responsibleai/ConceptBottleneck/CUB/CUB_200_2011/CUB_200_2011/image_class_labels.txt', 'r') as f:
    class_labels = np.loadtxt(f, dtype=int, usecols=[1])

# Get the attribute vector for the first image (index 0)
image_index = 148
attribute_vector = class_attribute_labels[class_labels[image_index] - 1]

# Binarize the attribute vector
binary_vector = (attribute_vector > 0.5).astype(int)

# Get the top 5 attributes with the highest values
top_indices = np.argsort(attribute_vector)[::-1][:5]
top_attributes = [attribute_names[i] for i in top_indices]

# Print the results
print('Attribute values:', binary_vector)
print('Top 5 attributes:', top_attributes)




Attribute values: [0 1 1 0 1 1 1 1 0 0 1 0 0 0 1 0 0 1 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 1 1 1
 0 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 0 1 0 1 0 0 0 0 1 1 1 0 1 1
 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1
 0 0 0 0 1 1 1 0 1 1 1 0 1 0 1 1 0 0 0 0 1 1 0 1 1 1 0 0 1 0 0 0 1 0 1 1 0
 1 1 1 1 0 1 0 1 0 1 1 0 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 1 0
 1 0 1 0 0 0 0 1 1 1 0 1 1 1 0 1 0 1 0 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1
 1 0 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 0 0 0 1
 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 0 1 0 0 0 0 1 1 1 0 0 0 1 0
 1 0 1 0 0 0 1 0 1 1 0 1 1 1 1 1]
Top 5 attributes: ['150 has_bill_length::about_the_same_as_head', '146 has_eye_color::black', '245 has_belly_pattern::solid', '290 has_bill_color::black', '241 has_tail_pattern::solid']


* There is a mismatch because the image inside each folder has a capital letter instead of having the same name of the folder. Idk how to fix it.
* I need to know the exact names of the images used in the saliency maps to extract the correct ground truth attributes. 

In [41]:
# Load the image file names
image_folder ='/Users/lucialarraona/Desktop/responsibleai/ConceptBottleneck/CUB/CUB_200_2011/CUB_200_2011/images/'
image_files = os.listdir(image_folder)

image_index = 0
species_folder = str(class_labels[image_index]).zfill(3) + '.' + class_names[image_index]
image_path = '/Users/lucialarraona/Desktop/responsibleai/ConceptBottleneck/CUB/CUB_200_2011/CUB_200_2011/images/{}/{}_{}.jpg'.format(species_folder, species_folder, str(image_index + 1).zfill(4))
image = Image.open(image_path)

# Display the image and print the results
plt.imshow(image)
plt.axis('off')
plt.title('Image with ID {}'.format(image_index + 1))
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/Users/lucialarraona/Desktop/responsibleai/ConceptBottleneck/CUB/CUB_200_2011/CUB_200_2011/images/000.001.Black_footed_Albatross/000.001.Black_footed_Albatross_0001.jpg'

# Quantitative Validation strategy

## Accuracy of explanations provided by each model

### Saliency map vs bounding boxes 

To calculate the accuracy using the saliency map and the ground truth explanation (bounding box), we can use the following steps:

To calculate the IoU, you can first convert the saliency map to a binary mask by thresholding it at a certain level (e.g., 0.5). Then, you can compute the intersection between the binary mask and the bounding box by multiplying them element-wise and summing the result. Similarly, you can compute the union between them by summing the number of pixels in both the binary mask and the bounding box and subtracting the intersection. Finally, you can calculate the IoU as the ratio of the intersection to the union.

The higher the IoU value, the better the saliency map aligns with the ground truth bounding box. This measure provides a way to evaluate the quality of saliency maps and how well they capture the object of interest in the image.

In [ ]:
# Path to saliency map
PATH_saliency_map = ''

In [70]:
# Path to ground truth bounding boxes 
ground_truth_bb = pd.read_csv('/Users/lucialarraona/Desktop/responsibleai/ConceptBottleneck/CUB/CUB_200_2011/CUB_200_2011/bounding_boxes.txt', header=None)
df= ground_truth_bb.rename(columns = { 0 : 'bbox'})

# split bbox column into separate columns
df[["index", "x1", "y1", "x2", "y2"]] = df["bbox"].str.split(" ", expand=True)

# convert values to appropriate data types
df["index"] = df["index"].astype(int)
df["x1"] = df["x1"].astype(float)
df["y1"] = df["y1"].astype(float)
df["x2"] = df["x2"].astype(float)
df["y2"] = df["y2"].astype(float)

df

,bbox,index,x1,y1,x2,y2
0,1 60.0 27.0 325.0 304.0,1,60.0,27.0,325.0,304.0
1,2 139.0 30.0 153.0 264.0,2,139.0,30.0,153.0,264.0
2,3 14.0 112.0 388.0 186.0,3,14.0,112.0,388.0,186.0
3,4 112.0 90.0 255.0 242.0,4,112.0,90.0,255.0,242.0
4,5 70.0 50.0 134.0 303.0,5,70.0,50.0,134.0,303.0
...,...,...,...,...,...,...
11783,11784 89.0 95.0 354.0 250.0,11784,89.0,95.0,354.0,250.0
11784,11785 157.0 62.0 184.0 219.0,11785,157.0,62.0,184.0,219.0
11785,11786 190.0 102.0 198.0 202.0,11786,190.0,102.0,198.0,202.0
11786,11787 3.0 20.0 408.0 307.0,11787,3.0,20.0,408.0,307.0


In [ ]:
def calculate_accuracy_iou(PATH_saliency_map, df):
    
    # Load the saliency map and ground truth bounding box
    saliency_map = cv2.imread(PATH_saliency_map, cv2.IMREAD_GRAYSCALE)
    x1, y1, x2, y2 = df['x1'],df['x2'],df['x3'],df['x4']  # example bounding box coordinates

    # Calculate the intersection and union areas
    intersection = np.sum(saliency_map[y1:y2, x1:x2] > 0)
    union = np.sum(saliency_map > 0) + (x2-x1)*(y2-y1) - intersection

    # Calculate the IoU
    iou = intersection / union
    
    return iou

### Concepts vs attributes

## Comprehensibility of explanations provided by each model

This could be assessed through a user study where participants are presented with explanations generated by each model and are asked to rate them based on how understandable they are. We can also use metrics such as reading time or error rates to evaluate the effectiveness of the explanations.

# Comparing the sucess of the two models

To compare the success of the two models, we can conduct a randomized controlled trial where participants are randomly assigned to either the saliency map or concept bottleneck group. Each group would be presented with the same set of input-output pairs and the explanations generated by their respective models. We can then collect data on accuracy and comprehensibility and compare the results between the two groups using statistical tests such as t-tests or ANOVA.